In [1]:
from pathlib import Path
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from functions.make_dataset import *
from functions.nn_pytorch import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader

In [2]:
path = '/Users/Martine Pedersen/Oppgaver/Project_3_FYSSTK/kagglehub/datasets/nikhil7280/weather-type-classification/versions/1/weather_classification_data.csv'
dataset = WeatherDataset(csv_file=path)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
    
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:

class WeatherNN(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        num_hidden_layers: int,
        output_dim: int,
        activation: str = "relu"
    ):
        super().__init__()

        # Choose activation function
        activations = {
            "relu": nn.ReLU(),
            "lrelu": nn.LeakyReLU(),
            "sigmoid": nn.Sigmoid(),
        }
        act = activations[activation.lower()]

        layers = []

        # Input layer
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(act)
        
        # Hidden layers
        for h in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(act)

        # Output layer
        layers.append(nn.Linear(hidden_dim, output_dim))
        layers.append(nn.Softmax())

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

    def train_model(self, train_loader, eta=1e-2, epochs=10):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=eta)

        for epoch in range(epochs):
            total_loss = 0.0
            for X, y in train_loader:
                logits = self(X)
                loss = criterion(logits, y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            #print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")
    
        
    def evaluate(self, loader):
        self.eval()  # set model to evaluation mode
        correct = 0
        total = 0

        with torch.no_grad():  # no gradient computation
            for X, y in loader:
                logits = self(X)                  # forward pass
                preds = logits.argmax(dim=1)     # predicted class
                correct += (preds == y).sum().item()
                total += y.size(0)

        return correct / total  # overall accuracy



In [ ]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader   = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = dataset.X.shape[1]  # number of features
hidden = 64
num_classes = len(dataset.encoders["Weather Type"].classes_)

model = WeatherNN(input_dim=input_dim, hidden_dim=hidden, num_hidden_layers=10, output_dim=num_classes, activation="relu")
cost_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

epochs = 10

In [19]:
model.train_model(train_loader, epochs=epochs)

accuracy = model.evaluate(test_loader)
print("Test accuracy:", accuracy)


Test accuracy: 0.2545454545454545


In [18]:
# Example: take one batch from the test_loader
X_batch, y_batch = next(iter(test_loader))

logits = model(X_batch)  # forward pass
print(logits)


tensor([[0.0000e+00, 9.3643e-27, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 4.3351e-24, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 9.9643e-18, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.0347e-22, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 9.6222e-22, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 4.5451e-21, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 4.1405e-27, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.3473e-22, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.9493e-22, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 2.3691e-23, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.8193e-24, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.4504e-23, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.9382e-16, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 8.2421e-17, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 2.6634e-26, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 8.9135e-27, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 6.5045e-22, 1.0000e+00, 0.0000e+00],
        [0.000

In [14]:
preds = logits.argmax(dim=1)
print(preds)

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


In [15]:
print("True labels:", y_batch[:10])
print("Predicted labels:", preds[:10])

True labels: tensor([3, 0, 2, 3, 1, 3, 3, 1, 1, 0])
Predicted labels: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


In [16]:
print(logits.sum(dim=1))  # should all be 1

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], grad_fn=<SumBackward1>)
